In [17]:
import pandas as pd

In [18]:
df = pd.read_csv(r'C:\Users\marku\Desktop\ML\MLGit\datasets\heart.csv')
df_XGB = df.copy()
df_RF = df.copy()
df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


# XGBoost

In [19]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

train_X, test_X, train_y, test_y = train_test_split(df_XGB.drop('output', axis=1), df_XGB['output'], random_state=40)

model = XGBClassifier(verbosity=0)
model.fit(train_X, train_y)

pred_y = model.predict(test_X)
preds = [round(value) for value in pred_y]

# evaluate predictions
accuracy = accuracy_score(test_y, preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 84.21%


C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


In [20]:
from sklearn.model_selection import GridSearchCV

XGBParam = {'eta': [0.01, 0.05, 0.1, 0.15, 0.2],
            'gamma': [0, 1, 10, 15],
            'max_depth': [3, 5, 6, 8, 10],
            'subsample': [0.5, 0.75, 1]}

XGB_grid = GridSearchCV(XGBClassifier(verbosity=0, tree_method='auto'), XGBParam)
XGB_grid.fit(train_X, train_y)

C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\marku\anaconda3\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):
C:\Users\marku\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when 

GridSearchCV(estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=None, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weight=None,
            

In [22]:
print(XGB_grid.best_score_)

0.8328502415458937


In [23]:
print(XGB_grid.best_params_)

{'eta': 0.1, 'gamma': 10, 'max_depth': 3, 'subsample': 0.5}


# RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

train_X, test_X, train_y, test_y = train_test_split(df_RF.drop('output', axis=1), df_RF['output'], random_state=40)

# Train and test random forest classifier with default parameters
model = RandomForestClassifier()
model.fit(train_X, train_y)

pred_y = model.predict(test_X)
preds = [round(value) for value in pred_y]

# evaluate predictions
accuracy = accuracy_score(test_y, preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
RFParam = {"max_features":['auto'],
           "max_depth":[2, 6, 12, 24, 48, 96],
           "n_estimators":[50, 100, 200, 300]}

RF_Grid = GridSearchCV(RandomForestClassifier(), RFParam)
RF_Grid.fit(train_X, train_y)

In [ ]:
print(RF_Grid.best_score_)

In [ ]:
print(RF_Grid.best_params_)